# #30 Results
<i>Guide on how access results</i>
***

Connect to LUSAS Modeller and check if a model is open

In [ ]:
from shared.LPI import *
lusas = get_lusas_modeller()

if not lusas.existsDatabase():
    raise Exception("A model must be open before running this code")

<div class="alert alert-block alert-warning">
<b>Note:</b> To successfully run the code below you must have a model solved.
</div>

<H2>1. Nodal Results</H2>

All nodes have displacement results. Here we simply loop through the nodes contained in the current selection and ask for the results of displacement and printing them

In [ ]:
selected_nodes : list[IFNode] = lusas.selection().getObjects("Node")
for n in selected_nodes:
    dx = n.getResults("Displacement", "DX")
    dy = n.getResults("Displacement", "DY")
    dz = n.getResults("Displacement", "DZ")
    print(dx, dy, dz)

Notice that in Python each of these returned values is a tuple containing the value of the displacement and a COM error code "(0x80020004) (-2147352572) Parameter not found"
The value of the result can be taken directly by modifying the code to take the first item of the returned tuple as follows:

<div class="alert alert-block alert-info">
<b>Note:</b> The missing value is the optional "loadcase" argument which is set as the loadcase providing the result when the active loadset is an envelope.
</div>

In [ ]:
selected_nodes : list[IFNode] = lusas.selection().getObjects("Node")
for n in selected_nodes:
    dx = n.getResults("Displacement", "DX")[0]
    dy = n.getResults("Displacement", "DY")[0]
    dz = n.getResults("Displacement", "DZ")[0]
    print(dx,dy,dz)

Not all results are available at a node. Reactions for example are only available at nodes with supports. If we modify the code above

In [ ]:
selected_nodes : list[IFNode] = lusas.selection().getObjects("Node")
for n in selected_nodes:
    fx = n.getResults("Reaction", "FX")[0]
    fy = n.getResults("Reaction", "FY")[0]
    fz = n.getResults("Reaction", "FZ")[0]
    print(fx,fy,fz)

Nodes without a support will return a value of 2.2250738585072014e-308. This is the smallest possible value represented by a 64bit double precision variable. 
We can avoid this small value by asking if a result is available

In [ ]:
selected_nodes : list[IFNode] = lusas.selection().getObjects("Node")
for n in selected_nodes:
    fx = n.getResults("Reaction", "FX")[0] if n.hasResults("Reaction", "FX") else 0
    fy = n.getResults("Reaction", "FY")[0] if n.hasResults("Reaction", "FY") else 0
    fz = n.getResults("Reaction", "FZ")[0] if n.hasResults("Reaction", "FZ") else 0
    print(fx,fy,fz)

So we could use this code to determine the total reactions of a selected group of nodes or indeed the whole model

In [ ]:
fx, fy, fz = 0, 0, 0
selected_nodes : list[IFNode] = lusas.selection().getObjects("Node")
for n in selected_nodes:
    fx += n.getResults("Reaction", "FX")[0] if n.hasResults("Reaction", "FX") else 0
    fy += n.getResults("Reaction", "FY")[0] if n.hasResults("Reaction", "FY") else 0
    fz += n.getResults("Reaction", "FZ")[0] if n.hasResults("Reaction", "FZ") else 0

print(f"Total reactions of selected nodes : {fx:.2f}, {fy:.2f}, {fz:.2f}")


The getResults function has 3 additional optional arguments as shown in the LPI Reference manual

`getResults(entity, component, [units], [loadcase], [context])`

The results we have been getting so far have been in model units and for the active loadset. 
The first additional argument allows us to ask for results in a different unit set.
The second additional argument is actually a return value that is used when the active loadset is an envelope. In this case the loadcase causing the requested results is returned through this argument.
The third argument allows us to specify other settings for the results such as a different loadset. We will return to resultsets later.

<H2>2. Element Results</H2>

Elements do not have a getResults function like nodes because elements have many different types of results. Shells, beams, solids and joint elements all have different results and result locations.

A beam element for example has results at each node, it also has a series of intermediate or "internal" results.
A shell element also nodal results at each node but it does not have internal results, instead it has additional results at the gauss/integration points 

The element interface therefore has several functions to deal with these differences and its up to you to call the correct one for the element you are looking at.

If we ask the database for all elements, that's exactly what we'll get and we'd have to write a lot of code to handle the various element types as follows:

In [ ]:
# Target the selected elements
targetElements : list[IFElement] = lusas.selection().getObjects("Element")
# Or uncomment this line to target all model elements
#targetElements : list[IFElement] = lusas.db().getObjects("Element")
for e in targetElements:
    stressType = e.getStressType()
    if e.getDomainDimension() == 1: # Beam Element
        if stressType == "Thick 3D Beam":
            for i in range(0, e.countInternalPoints()):
                my = e.getInternalResults(i, "Force/Moment - Thick 3D Beam", "My")[0]
                print(my)
    elif e.getDomainDimension() == 2: # Shell Element
        pass
    elif e.getDomainDimension() == 3: # Solid Element
        pass


The example above is limited to a single element type "Thick 3D Beam". Here we are retrieving the bending moment My for each internal point along the beam. We could get them as an array instead

In [ ]:
for e in targetElements:
    stressType = e.getStressType()
    if e.getDomainDimension() == 1: # Beam Element
        if stressType == "Thick 3D Beam":
            my = e.getInternalResultsArray("Force/Moment - Thick 3D Beam", "My")
            print(my)

The above can be repeated for all element types but you may have noticed that this approach is very slow and not recommended. A much better approach is to use results component sets.

<H2>3. Results Component Sets</H2>

A results component set is a container for a particular set of results. It is much more efficient than asking for results one by one.

In [ ]:
# Get the internal point results for all thick beam elements
results_my = lusas.database().getResultsComponentSet("Force/Moment - Thick 3D Beam", "My", "Internal")
i_my = results_my.getComponentNumber("My")

for e in targetElements:
    stressType = e.getStressType()
    if e.getDomainDimension() == 1: # Beam Element
        if stressType == "Thick 3D Beam":
            # Note the unitset is not optional but providing None uses the current database units
            my = results_my.getInternalResultsArray(i_my, e, None)
            print(my)

results_my = None # Make sure the notebook doesnt hang on to these results

The approach above still requires us to check all the elements. We can use another object to peform this operation once. The Object Set.

In [ ]:
# Get the internal point results for all thick beam elements
results_my = lusas.database().getResultsComponentSet("Force/Moment - Thick 3D Beam", "My", "Internal")
i_my = results_my.getComponentNumber("My")

# Create an object set containing only the thick 3d beam elements
beams = lusas.newObjectSet().add(targetElements).keep("Thick 3D Beam")
# or uncomment the following to get all 3D beam elements in the model
#beams = lusas.newObjectSet().add("Thick 3D Beam")

for e in beams.getObjects("Element"):
    # Note the unitset is not optional but providing None uses the current database units
    my = results_my.getInternalResultsArray(i_my, e, None)
    print(my)

results_my = None # Make sure the notebook doesnt hang on to these results
beams = None

Depending on what your model contains the above code should run much quicker than previous methods. This is because we filtered out only the elements of interest and removed any subsequent type checking.

The final piece of the puzzle is to provide a results context.

In [ ]:
# Create an object set containing only the thick 3d beam elements
beams = lusas.newObjectSet().add(targetElements).keep("Thick 3D Beam")

# Create a results context for the beam elements
context = lusas.newResultsContext(None)
context.getCalcResultsSet().add(beams)
context.setActiveLoadset(1)

# Get the internal point results for all thick beam elements
results_my = lusas.database().getResultsComponentSet("Force/Moment - Thick 3D Beam", "My", "Internal", context)
i_my = results_my.getComponentNumber("My")

for e in beams.getObjects("Element"):
    # Note the unitset is not optional but providing None uses the current database units
    my = results_my.getInternalResultsArray(i_my, e, None)
    print(my)

results_my = None # Make sure the notebook doesnt hang on to these results
context = None
beams = None

Here you can see that results context can be set to any loadcase and set of elements so we are no longer relying on the active settings of the user interface.

The general principle laid out above can be used for all elements, nodes and inspection location results.